# sciencedirect

In [1]:
from dotenv import load_dotenv
import requests
import os
import time

# Load environment variables
load_dotenv()
api_key = os.getenv("ELSEVIER_API_KEY")


def make_request(search_query: str, max_papers: int = 10) -> list:
    retries, start, count = 0, 0, 25
    results = []

    while len(results) < max_papers and start < 20000:
        params = {
            "query": search_query,
            "start": start,
            "count": count,
            "view": "COMPLETE",
        }

        headers = {
            "Accept": "application/json",
            "X-ELS-APIKey": api_key,
            "Content-Type": "application/json",
        }

        try:
            response = requests.get(
                "https://api.elsevier.com/content/search/scopus",
                params=params,
                headers=headers,
                timeout=10,
            )

            print(f"Status code: {response.status_code}")

            if response.status_code == 200:
                data = response.json()
                if "search-results" in data and "entry" in data["search-results"]:
                    results.extend(data["search-results"]["entry"])
                    print(
                        f"Retrieved {len(data['search-results']['entry'])} results. Total: {len(results)}"
                    )
                else:
                    print("No results found in the response.")
                    print(f"Response content: {response.text[:500]}...")
                    break
                start += count
            elif response.status_code == 429:
                wait_time = 2**retries
                print(f"Received 429, waiting {wait_time} seconds")
                time.sleep(wait_time)
                retries += 1
            elif response.status_code == 401:
                print(
                    f"Authentication failed. Please check your API key and permissions."
                )
                print(f"Response content: {response.text}")
                break
            else:
                print(f"Failed to retrieve results: {response.status_code}")
                print(f"Response content: {response.text}")
                break

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            break

    return results


# Test the function
results = make_request("machine learning")

# Process results if any were returned
if results:
    for doc in results:
        title = doc.get("dc:title", "N/A")
        doi = doc.get("prism:doi", "N/A")

        # Handle authors with more defensive coding
        authors = doc.get("authors", {})
        author_names = []

        if isinstance(authors, dict) and "author" in authors:
            author_list = authors["author"]
            if isinstance(author_list, list):
                author_names = [author.get("", "Unknown") for author in author_list]
            elif isinstance(author_list, dict):
                author_names = [author_list.get("", "Unknown")]

        formatted_authors = ", ".join(author_names) if author_names else "N/A"

        journal = doc.get("prism:publicationName", "N/A")
        date = doc.get("prism:coverDate", "N/A")

        # Handle links with more defensive coding
        links = doc.get("link", [])
        paper_link = "N/A"

        if isinstance(links, list):
            for link in links:
                if isinstance(link, dict) and "@href" in link:
                    paper_link = link["@href"]
                    break

        free = doc.get("openaccess", "N/A")

        print(f"Title: {title}")
        print(f"DOI: {doi}")
        print(f"Authors: {formatted_authors}")
        print(f"Journal: {journal}")
        print(f"Date: {date}")
        print(f"Paper Link: {paper_link}")
        print(f"Open Access: {free}")
        print("----" * 10)
else:
    print("No results were returned.")

Status code: 401
Authentication failed. Please check your API key and permissions.
Response content: {"service-error":{"status":{"statusCode":"AUTHORIZATION_ERROR","statusText":"The requestor is not authorized to access the requested view or fields of the resource"}}}
No results were returned.


# scopus

In [ ]:
def make_request(search_query: str, max_papers: int = 20000) -> list:
    retries, start, count = 0, 0, 25
    results = []
    while len(results) < max_papers and start < 20000:
        params = {
            "query": search_query,
            "start": start,
            "count": count,
            "view": "COMPLETE",
        }

        headers = {"Accept": "application/json", "X-ELS-APIKey": api_key}

        response = requests.get(
            "https://api.elsevier.com/content/search/scopus",
            params=params,
            headers=headers,
            timeout=10,
        )
        print(f"Status code: {response.status_code}")
        if response.status_code == 200:
            data = response.json()
            if "search-results" in data and "entry" in data["search-results"]:
                results.extend(data["search-results"]["entry"])
                print(
                    f"Retrieved {len(data['search-results']['entry'])} results. Total: {len(results)}"
                )
            else:
                print("No results found in the response.")
                break
            start += count
        elif response.status_code == 429:
            wait_time = 2**retries
            print(f"Received 429, waiting {wait_time}")
            time.sleep(wait_time)
            retries += 1
        else:
            print(f"Failed to retrieve results: {response.status_code}")
            break

    return results

In [5]:
from dotenv import load_dotenv
import requests
import os
import csv
import time
from pprint import pprint

load_dotenv()

api_key = os.getenv("ELSEVIER_API_KEY")


# --- Scopus --- #
def get_request(api_key: str, query: str) -> dict:
    headers = {
        "Accept": "application/json",
        "X-ELS-APIKey": api_key,
        "view": "STANDARD",
    }

    url = "https://api.elsevier.com/content/search/sciencedirect"
    params = {"query": query, "offset": 0}

    response = requests.get(url, params=params, headers=headers)
    return response.json()


data = get_request(api_key, query="microbiology")


for idx, doc in enumerate(data):
    # Process each document as needed
    pprint(data)

    # print(f"""{idx + 1}""")
    # print(f"\nPaper Link: {doc['link'][2]['@href']}")
    # print(f"\nPaper Author: {doc.get('dc:creator', '')}")
    # print(f"\nPaper DOI: {doc.get('prism:doi', '')}")
    # print(f"\nAffiliation: {doc.get('affiliation', [{}])[0].get('affilname', '')}")
    # print(f"\nPublication Name: {doc.get('prism:publicationName', '')}")
    # print(f"\nPublish Date: {doc.get('prism:coverDate', '')}")
    # print("----" * 30)
    # break

{'service-error': {'status': {'statusCode': 'AUTHORIZATION_ERROR',
                              'statusText': 'The requestor is not authorized '
                                            'to access the requested view or '
                                            'fields of the resource'}}}


In [2]:
from dotenv import load_dotenv
import requests
import os
import csv
import time
from pprint import pprint

load_dotenv()

api_key = os.getenv("ELSEVIER_API_KEY")

params = {
    "query": "microbiology",
    "count": 10,
}

headers = {"Accept": "application/json", "X-ELS-APIKey": api_key}

response = requests.get(
    "https://api.elsevier.com/content/search/scopus",
    params=params,
    headers=headers,
    timeout=10,
)
response.json()

{'search-results': {'opensearch:totalResults': '5015857',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '10',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'microbiology',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=10&query=microbiology',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=10&query=microbiology',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'next',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=10&count=10&query=microbiology',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'last',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=4990&count=10&query=microbiology',
    '@type': 'application/json'}],
  'entry': [{'@_fa': 'true',
    'link': [{'@_fa': 'true',
      '@r